In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("OpenSound/CapTTS-SFT")
df = ds["train"].to_pandas()
df

/home/nhandt23/anaconda3/envs/speechcraft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,source,audio_path,caption,text,speech_duration,accent,emotion,speaking_rate,intrinsic_tags,emotion_text_prompt,all_tags,age,pitch,situational_tags,gender,basic_tags,noise,speech_monotony
0,vctk,p345_260.wav,An American adult male voice with a dark tone.,What is so unusual about this case?,3.913917,american,None,None,None,None,None,None,None,None,None,None,None,None
1,vctk,p271_197.wav,"A mature man's Scottish voice, masculine and g...",Treasury spokesman Malcolm Bruce is also consi...,5.364458,scottish,None,None,None,None,None,None,None,None,None,None,None,None
2,vctk,p333_173.wav,"A mature American female voice, bright and sli...","Not everyone, however, was pleased with the pr...",3.800917,american,None,None,None,None,None,None,None,None,None,None,None,None
3,vctk,p252_017.wav,A mature man's Scottish voice.,Others have tried to explain the phenomenon ph...,3.862500,scottish,None,None,None,None,None,None,None,None,None,None,None,None
4,vctk,p335_231.wav,"An adult female voice, sounding weak, with a d...","They scored a goal, but we have to keep playing.",4.154125,newzealand,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347778,libritts-r,train-other-500/985/126228/985_126228_000056_0...,"With a slightly relaxed cadence, his words flo...",phillotson seemed not to notice to be surround...,6.160000,None,None,moderate speed,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347779,libritts-r,train-other-500/985/126228/985_126228_000056_0...,The middle-aged man's voice is a unique blend ...,as soon as they had signed their names and c...,5.440000,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347780,libritts-r,train-other-500/985/126228/985_126228_000057_0...,"His speech, while not hurried, is far from let...",the meal at his lodging was a very simple affa...,4.160000,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347781,libritts-r,train-other-500/985/126228/985_126228_000057_0...,"With a subtle hint of dark humor, his voice is...",in crossing the pavement to the fly she looked...,4.680000,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone


In [2]:
df["source"].unique()

array(['vctk', 'emovoice-db', 'voxceleb', 'expresso', 'ears',
       'libritts-r'], dtype=object)

## Expresso

In [3]:
import pandas as pd
import os

base_dir_read = '/data1/speech/nhandt23/06_binh/dataset/expresso/audio_48khz/read/'

is_conversational = (df['source'] == 'expresso') & \
                    (df['audio_path'].str.split('_').str[0].str.contains('-'))

df_clean = df[~is_conversational].copy()

def get_read_path(filename):
    try:
        parts = filename.split('_')
        speaker = parts[0] # ex01
        emotion = parts[1] # happy
        # Ghép: base_dir / speaker / emotion / base / filename
        return os.path.join(base_dir_read, speaker, emotion, 'base', filename)
    except:
        return filename

df_clean['audio_path'] = df_clean.apply(
    lambda row: get_read_path(row['audio_path']) if row['source'] == 'expresso' else row['audio_path'], 
    axis=1
)
print(f"Số lượng dòng ban đầu: {len(df)}")
print(f"Số lượng dòng sau khi lọc: {len(df_clean)}")
""

Số lượng dòng ban đầu: 347783
Số lượng dòng sau khi lọc: 340670


''

## LibriTTS-R

In [4]:
libri_dir = "/data1/speech/cuongph/01_database/librittsr/LibriTTS_R"
df_clean["audio_path"] = df_clean.apply(
    lambda row: os.path.join(libri_dir, row["audio_path"]) if row["source"] == "libritts-r" else row["audio_path"],
    axis=1
)

## VoxCeleb

In [5]:
import os
from tqdm import tqdm

voxceleb_dir2 = "/data1/speech/kienpt/data/speaker_verify/train_VoxCeleb2/wavs"
voxceleb_dir1 = "/data1/speech/speechData/rawData/Public_Data/VOXCELEB1/wav"
rm_idx = []

target_df = df_clean[df_clean["source"] == "voxceleb"]

for idx, row in tqdm(target_df.iterrows(), total=target_df.shape[0]):
    audio_path = row["audio_path"] # Ví dụ: id10230-WxmrMgdkqOw-00004.wav
    parts = audio_path.split("-")
    
    # parts[0] = id10230
    # parts[1] = WxmrMgdkqOw
    # parts[2] = 00004.wav (Đã có sẵn .wav nếu tên file gốc có)
    
    filename = parts[2]
    if not filename.endswith(".wav"):
        filename += ".wav"
        
    wavs_path2 = os.path.join(voxceleb_dir2, parts[0], parts[1], filename)
    wavs_path1 = os.path.join(voxceleb_dir1, parts[0], parts[1], filename)
    
    if not os.path.exists(wavs_path2) and not os.path.exists(wavs_path1):
        if len(rm_idx) < 3:
            print(f"Missing: {audio_path}")
        rm_idx.append(idx)

print(f"Tổng số file thiếu: {len(rm_idx)}")
df_clean = df_clean.drop(rm_idx)
df_clean = df_clean.reset_index(drop=True)

  0%|          | 199/89520 [00:00<01:23, 1070.23it/s]

Missing: id10270-GWXujl-xAVM-00033.wav
Missing: id04845-GN8pKT-NlQ4-00116.wav
Missing: id04423-E-mChLO917s-00133.wav


100%|██████████| 89520/89520 [00:34<00:00, 2606.71it/s]


Tổng số file thiếu: 16111


In [6]:
import os

voxceleb_dir = "/data1/speech/kienpt/data/speaker_verify/train_VoxCeleb2/wavs"

def make_absolute_path(row):
    if row["source"] == "voxceleb":
        parts = row["audio_path"].split("-")
        if len(parts) >= 3:
            filename = parts[2]
            if not filename.endswith(".wav"):
                filename += ".wav"
            return os.path.join(voxceleb_dir, parts[0], parts[1], filename)
    
    return row["audio_path"]

df_clean["audio_path"] = df_clean.apply(make_absolute_path, axis=1)

print("Sample path sau khi update:")
print(df_clean[df_clean["source"] == "voxceleb"]["audio_path"].iloc[0])

Sample path sau khi update:
/data1/speech/kienpt/data/speaker_verify/train_VoxCeleb2/wavs/id10230/WxmrMgdkqOw/00004.wav


## ears

In [7]:
import os

ears_path = "/data1/speech/nhandt23/06_thang/ears_dataset"

def make_absolute_path(row):
    if row["source"] == "ears":
        parts = row["audio_path"].split("-")
        return os.path.join(ears_path, parts[0], parts[1])
    
    return row["audio_path"]

df_clean["audio_path"] = df_clean.apply(make_absolute_path, axis=1)

print("Sample path sau khi update:")
print(df_clean[df_clean["source"] == "ears"]["audio_path"].iloc[0])

Sample path sau khi update:
/data1/speech/nhandt23/06_thang/ears_dataset/p059/rainbow_07_lowpitch.wav


## Emovoice-db

In [8]:
df_clean[df_clean["source"] == "emovoice-db"].iloc[0]["audio_path"]

'gpt4o_11110_surprised_ash.wav'

In [9]:
emodb_path = "/data1/speech/speechData/rawData/Public_Data/EmoVoice-DB/audio"

df_clean["audio_path"] = df_clean.apply(
    lambda row: os.path.join(emodb_path, row["audio_path"]) if row["source"] == "emovoice-db" else row["audio_path"],
    axis=1
)

## vctk

In [10]:
len(df_clean[df_clean["source"] == "vctk"])

23677

In [11]:
import os
vctk_path = "/data1/speech/speechData/rawData/Public_Data/VCTK/VCTK-Corpus/wav48"

def make_absolute_path(row):
    if row["source"] == "vctk":
        parts = row["audio_path"].split("_")
        return os.path.join(vctk_path, parts[0], row["audio_path"])
    
    return row["audio_path"]

df_clean["audio_path"] = df_clean.apply(make_absolute_path, axis=1)

print("Sample path sau khi update:")
print(df_clean[df_clean["source"] == "vctk"]["audio_path"].iloc[0])

Sample path sau khi update:
/data1/speech/speechData/rawData/Public_Data/VCTK/VCTK-Corpus/wav48/p345/p345_260.wav


In [12]:
# check

def check_exist(path):
    return os.path.exists(path)

df_clean["exists"] = df_clean["audio_path"].apply(check_exist)
print(df_clean[df_clean["exists"] == 0].groupby("source").size())
df_clean = df_clean[df_clean["exists"] != 0].reset_index(drop=True)

source
vctk          324
voxceleb    18968
dtype: int64


In [13]:
len(df_clean)

305267

## Export

In [14]:
df_clean.head()

,source,audio_path,caption,text,speech_duration,accent,emotion,speaking_rate,intrinsic_tags,emotion_text_prompt,all_tags,age,pitch,situational_tags,gender,basic_tags,noise,speech_monotony,exists
0,vctk,/data1/speech/speechData/rawData/Public_Data/V...,An American adult male voice with a dark tone.,What is so unusual about this case?,3.913917,american,None,None,None,None,None,None,None,None,None,None,None,None,True
1,vctk,/data1/speech/speechData/rawData/Public_Data/V...,"A mature man's Scottish voice, masculine and g...",Treasury spokesman Malcolm Bruce is also consi...,5.364458,scottish,None,None,None,None,None,None,None,None,None,None,None,None,True
2,vctk,/data1/speech/speechData/rawData/Public_Data/V...,"A mature American female voice, bright and sli...","Not everyone, however, was pleased with the pr...",3.800917,american,None,None,None,None,None,None,None,None,None,None,None,None,True
3,vctk,/data1/speech/speechData/rawData/Public_Data/V...,A mature man's Scottish voice.,Others have tried to explain the phenomenon ph...,3.862500,scottish,None,None,None,None,None,None,None,None,None,None,None,None,True
4,vctk,/data1/speech/speechData/rawData/Public_Data/V...,"An adult female voice, sounding weak, with a d...","They scored a goal, but we have to keep playing.",4.154125,newzealand,None,None,None,None,None,None,None,None,None,None,None,None,True


In [18]:
df_clean[["audio_path","text" ,"caption"]].to_csv("final_data.txt", sep="|", header=None, index=False)